In [1]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import os
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')

#execution time for this notebook is about 110 min with a computer dealing with 12 threads, 24Go RAM and broadband access

In [2]:
pd.options.display.max_columns = None

# Dataset prior to july 2021

In [3]:
df1 = pd.read_csv('../src/dpe-69.csv')

In [4]:
# the dataset contains all the diagnostics of the Rhône area

df1.shape

(332834, 131)

In [5]:
df1.head()

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage
0,134,1369V1000001F,227,2,1,3CL - DPE,V2012,NaN,NaN,2013-04-10,2013-04-10,2011-08-15,Logement non occupé depuis plus d'une année. A...,NaN,416.56,F,21.73,D,1,Habitation (Maisons individuelles),NaN,NaN,1980,100.71,100.71,69,DÉCINES-CHARPIEU,50,rue,de Verdun,28,NaN,NaN,NaN,NaN,69150,69275,69275,sans objet,NaN,NaN,NaN,2.0,NaN,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Socotec certification,"3, av du Centre 78280 Guyancourt",0.0,0,2013-04-10 04:00:00,3.480600,46.828209,0.73,poi.bus_stop,Avenue de Verdun / Rue des Pendants (Arrêt de ...,https://osm.org/node/1804339871,NaN,NaN,V1,1,6.1,Pour les bâtiments à usage principal d’h...,1_V1_DPE_6.1.pdf,0,Vente,Vente,1,TR002_001,Maison Individuelle,Maison,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,69 - Rhône,1,3,0,1200,4900,54000,80,340.0,1.5,NaN,-10,392.0,25.2,53.5,67.8,90.0,29.0,43.0,H1,6.58,10.5,42030,3.6,Ec,4,6,3.0,4
1,147,1369L1000001E,3825,3,9,3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-09,2013-04-10,2011-09-15,NaN,NaN,254.00,E,47.00,E,1,Maison Individuelle,NaN,NaN,1978,97.00,97.00,69,COURS-LA-VILLE,NaN,NaN,13 

In [6]:
# extracting only the year in the date of the diagnostic

df1['annee'] = df1['date_etablissement_dpe'].apply(lambda x: str(x).split('-')[0])

In [7]:
print(df1.columns.tolist())

['id', 'numero_dpe', 'usr_diagnostiqueur_id', 'usr_logiciel_id', 'tr001_modele_dpe_id', 'nom_methode_dpe', 'version_methode_dpe', 'nom_methode_etude_thermique', 'version_methode_etude_thermique', 'date_visite_diagnostiqueur', 'date_etablissement_dpe', 'date_arrete_tarifs_energies', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'consommation_energie', 'classe_consommation_energie', 'estimation_ges', 'classe_estimation_ges', 'tr002_type_batiment_id', 'secteur_activite', 'tr012_categorie_erp_id', 'tr013_type_erp_id', 'annee_construction', 'surface_habitable', 'surface_thermique_lot', 'tv016_departement_id', 'commune', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'portee_dpe_batiment', 'partie_batiment', 'shon', 'surface_utile', 'surface_thermique_parti

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332834 entries, 0 to 332833
Columns: 132 entries, id to annee
dtypes: float64(49), int64(28), object(55)
memory usage: 335.2+ MB


In [9]:
df1.describe(include='all')

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage,annee
count,3.328340e+05,332834,332834.000000,332834.000000,332834.000000,332834,251448,55934,52632,332834,332834,332834,53090,29501,3.328340e+05,332834,3.328340e+05,332834,332834.000000,26224,2047.000000,16443.000000,332834.000000,332828.000000,332828.000000,332834.0,332614,117626,72243,332754,92482,26055,100146,47120,8884,332768,332768.0,332768.0,159813,11895,0.0,45027.000000,267208.000000,45,250526.000000,267208.000000,267208.000000,250526.0,250526.000000,267208.000000,250526.0,250526.0,267208.000000,250526.0,0.0,250526.0,250531.0,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,332834.0,331746,319712,307844.000000,332834.0,332834,310283.000000,310283.000000,332834.000000,310283,310283,306909,300444,829,332834,332834.000000,332834,332834,332834,332834.0,332834,332834,332834.000000,332834,332834,332834,332834.0,332834.000000,332834.000000,2047,2047,2047,2047.0,16443,16443,16443.000000,16443.0,16443,332834.0,332834,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,0.0,332834.0,332834.0,3.328340e+05,332834.0,3.328340e+05,332

In [10]:
# official codes for the "arrondissements" of Lyon city

list_arrondissements = range(69381,69390,1)

In [11]:
# keeping only Lyon city and its arrondissements
dfi1 = pd.DataFrame()
mask = df1['code_insee_commune']==69123
df_temp = df1[mask]
dfi1 = pd.concat([dfi1, df_temp])
dfi1.shape
for i in list_arrondissements :
    mask = df1['code_insee_commune']==i
    df_temp = df1[mask]
    dfi1 = pd.concat([dfi1, df_temp])
print(dfi1.shape)

(94331, 132)


In [12]:
# keeping only the columns that interest us

col_to_keep = ['annee', 'classe_consommation_energie', 'secteur_activite', 'annee_construction', 'surface_habitable', 'arrondissement', 'type_voie', 'nom_rue', 'code_insee_commune', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'nombre_niveaux', 'surface_baies_orientees_nord', 'surface_baies_orientees_est_ouest', 'surface_baies_orientees_sud', 'longitude', 'latitude', 'tr001_modele_dpe_type_libelle']

In [13]:
dfi1 = dfi1[col_to_keep]
dfi1.shape

(94331, 21)

In [14]:
# looking at the values for the diagnostics
dfi1['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'N', 'B'], dtype=object)

In [15]:
# 'N' value is not in the normal range of the results (which are A to G), removing 'N' rows
mask = dfi1['classe_consommation_energie']=='N'
dfi1 = dfi1[~mask]
dfi1.shape

(77626, 21)

In [16]:
dfi1['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'B'], dtype=object)

In [17]:
dfi1['secteur_activite'].unique()

array(['Habitation (parties privatives d immeuble col', nan,
       "Habitation (Parties privatives d'immeuble col",
       'Habitation (maisons individuelles)', 'Bureaux',
       'Habitation (Maisons individuelles)', 'Local commercial', 'Garage',
       'Residence', 'Habitation (Parties communes)', 'Commerce',
       'Habitation (parties communes)', 'Autres',
       "Immeuble d'habitation collectif",
       'Batiment(s) a usage de commerces et d habitat',
       'Batiment a usage de bureaux et d habitation',
       'Etablissements sociaux',
       'Habitation (maison en copropriete horizontale',
       'Chambres privatives', 'Appartement', 'Etablissement sociaux',
       'Habitation (maison individuelle)', 'Maison individuelle',
       'Centre commercial'], dtype=object)

In [18]:
# keep only diagnostics related to flats and houses
activite_to_dump = ['Bureaux',
       'Local commercial', 'Garage',
       'Commerce',
       'Autres',
       'Etablissements sociaux',
       'Chambres privatives', 'Etablissements sociaux', 'Etablissement sociaux', 'Centre commercial']

In [19]:
dfi1['mask_activite'] = dfi1['secteur_activite'].map(lambda x : 0 if x in activite_to_dump else 1)
mask = dfi1['mask_activite']==1
dfi1 = dfi1[mask]
dfi1.shape

(77602, 22)

In [20]:
# keep only diagnostics done for sales
dfi1['tr001_modele_dpe_type_libelle'].unique()

array(['Location', 'Copropriété', 'Vente', 'Neuf', 'Bâtiment public'],
      dtype=object)

In [21]:
mask = dfi1['tr001_modele_dpe_type_libelle'] == 'Vente'
dfi1 = dfi1[mask]
dfi1.shape

(31168, 22)

In [22]:
dfi1['code_insee_commune'] = dfi1['code_insee_commune'].astype(int)

In [23]:
dfi1['code_insee_commune'].unique()

array([69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388,
       69389])

In [24]:
#determine orientation by taking the max of 'surface_baies_orientees_nord', 'surface_baies_orientees_est_ouest', 'surface_baies_orientees_sud'
def orientation(n, eo, s):
    try :
        maxi = max(n, eo, s)
    except :
        return None

    if maxi == 0 :
        return None
    if maxi == n :
        return 'N'
    if maxi == eo :
        return 'EO'
    if maxi == s :
        return 'S'
    


In [25]:
dfi1['orientation'] = list(map(orientation, dfi1['surface_baies_orientees_nord'], dfi1['surface_baies_orientees_est_ouest'], dfi1['surface_baies_orientees_sud']))

In [26]:
dfi1['orientation'].value_counts()

EO    7500
S     3835
N     3410
Name: orientation, dtype: int64

In [27]:
dfi1['orientation'].isna().sum()

16423

In [28]:
dfi1['annee'] = pd.to_numeric(dfi1['annee'])
dfi1['annee'] = dfi1['annee'].astype(int)

In [29]:
#determine code postal in order to use the API for getting the geokeys
list_keys = [69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388,69389]
list_values = [69000, 69001, 69002, 69003, 69004, 69005, 69006, 69007, 69008,69009]
dict_code = dict(zip(list_keys, list_values))
def code_postal(c) :
    return dict_code[c]

In [30]:
dfi1['code_postal'] = dfi1['code_insee_commune'].apply(code_postal)

In [31]:
#create column 'rue' to gather type_voie and nom_rue
def rue(type, nom) :
    if pd.notnull(type) and pd.notnull(nom) :
        return type + ' ' + nom
    if pd.notnull(type)==False :
        return nom
    if pd.notnull(nom)==False :
        return type

In [32]:
dfi1['type_voie'] = dfi1['type_voie'].astype(str)
dfi1['nom_rue'] = dfi1['nom_rue'].astype(str)
dfi1['rue'] = list(map(rue, dfi1['type_voie'], dfi1['nom_rue'])) # concatenate and removing nan

In [33]:
dfi1['etage'].unique()

array(['1er', '3eme Etage', '1er Etage', 'NC', '2eme Etage', nan,
       '1er Etage sur Entresol', 'RDC', '4eme Etage', '5eme Etage',
       '6eme Etage', '1', '5ème', '6', 'RDC ', '03', '2', '04', 'rdc ',
       '3ème étage', '02', 'Rez de jardin', '9eme Etage',
       'Rez de chaussee', '3ème', '1ER', '10eme Etage', '2ème', '4ème',
       '1er SS', '8', '07', '4', '06', '5ème étage', '05', '01', '15',
       '1er étage', '2ème ', '2ème étage', 'Rez de chaussée', '4ème ',
       '4ème étage', '3', '12ème', '7eme Etage', '8eme Etage', '4eme',
       '2ème étage sur entresol', '6ème', 'RDJ', '2eme', '10', '7ème',
       '3eme', '8ème', '2ème étage SUR ENTRESOL', 'ENTRE', '08', '5',
       'Rez de chaussee sureleve', '7', '+ 3.5', '12eme Etage', '7eme',
       '6ème étage', 'Entresol', 'RRDJ', '5eme', '6eme',
       '4eme Etage sur Entresol', '14eme Etage', 'rdc', 'Non communiqué',
       '5eme et 6eme Etage', 'RDC G', 'RDC D', '1ER G', '2G', '-1', 'R+1',
       '12', '3eme et 4eme Etage

In [34]:
#getting the floor numbers, which are badly written inside the file. Checking keywords and isolating numbers
dfi1['Etage_num'] = dfi1['etage'].apply(lambda x: list(map(int, re.findall(r'\d+', str(x))))) # isolate numbers
dfi1['RDC'] = dfi1['etage'].apply(lambda x: re.findall(r"rdc|rez de chaussee", str(x).lower())) # find rez de chaussee

In [35]:
def etage1(num, RDC) :
    if len(RDC) > 0 :    # if RDC is detected, then the floor is 'rez de chaussee'
        return 0
    if len(num) > 0 : # if we get at least one number, we assume we take the first one
        return num[0]
    else :
        return float('NaN')

In [36]:
dfi1['etage'] = list(map(etage1, dfi1['Etage_num'], dfi1['RDC']))

In [37]:
dfi1.describe(include='all')

,annee,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,code_insee_commune,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite,orientation,code_postal,rue,Etage_num,RDC
count,31168.000000,31168,3042,31168.000000,31168.000000,4841,31168,31168,31168.000000,6721,760,16536,3422.000000,844,30350.000000,28828.000000,28828.000000,28828.000000,30907.000000,30907.000000,31168,31168.0,14745,31168.000000,31168,31168,31168
unique,NaN,7,12,NaN,NaN,25,116,20323,NaN,709,154,3345,NaN,245,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,3,NaN,21037,21,3
top,NaN,D,Habitation (parties privatives d immeuble col,NaN,NaN,00,nan,Jules VERNE,NaN,20,NC,Etage 1,NaN,Gauche,NaN,NaN,NaN,NaN,NaN,NaN,Vente,NaN,EO,NaN,rue Jules VERNE,[],[]
freq,NaN,12011,2239,NaN,NaN,822,24804,154,NaN,244,311,1224,NaN,176,NaN,NaN,NaN,NaN,NaN,NaN,31168,NaN,7500,NaN,153,28113,30796
mean,2017.345547,NaN,NaN,1848.572992,113.215006,NaN,NaN,NaN,69174.087076,NaN,NaN,NaN,2.943892,NaN,1.150214,1.680280,3.060188,1.571181,4.842422,45.757101,NaN,1.0,NaN,69001.027817,NaN,NaN,NaN
std,2.156311,NaN,NaN,471.613210,465.656422,NaN,NaN,NaN,103.877582,NaN,NaN,NaN,2.150792,NaN,1.666522,33.785754,10.173962,5.526032,0.052073,0.040443,NaN,0.0,NaN,2.347644,NaN,NaN,NaN
min,2000.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.368634,43.663685,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
25%,2016.000000,NaN,NaN,1947.000000,46.820000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,1.000000,NaN,1.000000,0.000000,0.000000,0.000000,4.827331,45.746387,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
50%,2018.000000,NaN,NaN,1960.000000,66.450000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,3.000000,NaN,1.000000,0.000000,0.000000,0.000000,4.844901,45.755725,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN
75%,2019.000000,NaN,NaN,1989.000000,87.000000,NaN,NaN,NaN,69123.000000,NaN,NaN,NaN,4.000000,NaN,1.000000,0.000000,4.160000,0.000000,4.863324,45.768485,NaN,1.0,NaN,69000.000000,NaN,NaN,NaN


In [38]:
# looking at the number of diagnostics each year to check if it is normal
dfi1.groupby(['annee']).size()

annee
2000       2
2012       3
2013     691
2014    3859
2015    2789
2016    3659
2017    4308
2018    4485
2019    4989
2020    5598
2021     785
dtype: int64

In [39]:
# reducing dataset by keeping useful columns only
list_to_keep = ['annee', 'classe_consommation_energie','secteur_activite', 'annee_construction', 'surface_habitable', 'numero_rue', 'rue', 'code_postal', 'etage', 'nombre_niveaux', 'orientation']
dfi1 = dfi1[list_to_keep]

In [40]:
# giving standard names to the columns
list_columns_dpe = ['annee_dpe', 'diag_dpe','type_bat_dpe', 'annee_construction_dpe', 'surface_habitable_dpe', 'num_rue_dpe', 'rue_dpe', 'code_postal_dpe', 'etage_dpe', 'nbe_niveaux_dpe', 'orientation_dpe']

In [41]:
dfi1.columns = list_columns_dpe

In [43]:
# the first dataset is ready
dfi1.head()

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
4143,2013,G,Habitation (Parties privatives d'immeuble col,1995,19.28,8,rue de l'Epargne,69000,1.0,0.0,None
4206,2013,E,Habitation (parties privatives d immeuble col,1974,68.00,29,RUE RENE LEYNAUD,69000,3.0,1.0,None
4254,2013,E,Habitation (parties privatives d immeuble col,2000,32.00,27,RUE DES GRANGES,69000,1.0,1.0,S
4262,2013,D,NaN,2000,38.23,NC,nan 60 Boulevard Pinel Bâtiment B,69000,NaN,1.0,S
4268,2013,E,Habitation (parties privatives d immeuble col,1974,66.11,137,RUE de Montagny,69000,2.0,1.0,None


# Dataset after july 2021, without new accomodation

In [45]:
df2 = pd.read_csv('../src/dpe-lyon-ancien-21.csv') # dataset for only Lyon city

In [46]:
df2.shape

(36746, 246)

In [47]:
df2.head()

,Unnamed: 0,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),Méthode_application_DPE,N°_immatriculation_copropriété,Invariant_fiscal_logement,N°_RPLS_logement,Etiquette_GES,Etiquette_DPE,Année_construction,Type_bâtiment,Type_installation_chauffage,Type_installation_ECS_(général),Période_construction,Hauteur_sous-plafond,Nombre_appartement,Nombre_niveau_immeuble,Nombre_niveau_logement,Surface_habitable_immeuble,Surface_habitable_logement,Surface_tertiaire_immeuble,Classe_inertie_bâtiment,Typologie_logement,Position_logement_dans_immeuble,Classe_altitude,Zone_climatique_,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Complément_d'adresse_bâtiment,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Conso_5_usages_é_finale,Conso_5_usages/m²_é_finale,Conso_chauffage_é_finale,Conso_chauffage_dépensier_é_finale,Conso_éclairage_é_finale,Conso_ECS_é_finale,Conso_ECS_dépensier_é_finale,Conso_refroidissement_é_finale,Conso_refroidissement_dépensier_é_finale,Conso_auxiliaires_é_finale,Conso_5_usages_é_primaire,Conso_5_usages_par_m²_é_primaire,Conso_chauffage_é_primaire,Conso_chauffage_dépensier_é_primaire,Conso_éclairage_é_primaire,Conso_ECS_é_primaire,Conso_ECS_dépensier_é_primaire,Conso_refroidissement_é_primaire,Conso_refroidissement_dépensier_é_primaire,Conso_auxiliaires_é_primaire,Emission_GES_5_usages,Emission_GES_5_usages_par_m²,Emission_GES_chauffage,Emission_GES_chauffage_dépensier,Emission_GES_éclairage,Emission_GES_ECS,Emission_GES_ECS_dépensier,Emission_GES_refroidissement,Emission_GES_refroidissement_dépensier,Emission_GES_auxiliaires,Conso_5_usages_é_finale_énergie_n°1,Conso_chauffage_é_finale_énergie_n°1,Conso_ECS_é_finale_énergie_n°1,Coût_total_5_usages_énergie_n°1,Coût_chauffage_énergie_n°1,Coût_ECS_énergie_n°1,Emission_GES_5_usages_énergie_n°1,Emission_GES_chauffage_énergie_n°1,Emission_GES_ECS_énergie_n°1,Type_énergie_n°1,Conso_5_usages_é_finale_énergie_n°2,Conso_chauffage_é_finale_énergie_n°2,Conso_ECS_é_finale_énergie_n°2,Coût_total_5_usages_énergie_n°2,Coût_chauffage_énergie_n°2,Coût_ECS_énergie_n°2,Emission_GES_5_usages_énergie_n°2,Emission_GES_chauffage_énergie_n°2,Emission_GES_ECS_énergie_n°2,Type_énergie_n°2,Conso_5_usages_é_finale_énergie_n°3,Conso_chauffage_é_finale_énergie_n°3,Conso_ECS_é_finale_énergie_n°3,Coût_total_5_usages_énergie_n°3,Coût_chauffage_énergie_n°3,Coût_ECS_énergie_n°3,Emission_GES_5_usages_énergie_n°3,Emission_GES_chauffage_énergie_n°3,Emission_GES_ECS_énergie_n°3,Type_énergie_n°3,Coût_total_5_usages,Coût_chauffage,Coût_chauffage_dépensier,Coût_éclairage,Coût_ECS,Coût_ECS_dépensier,Coût_refroidissement,Coût_refroidissement_dépensier,Coût_auxiliaires,Logement_traversant_(0/1),Présence_brasseur_air_(0/1),Indicateur_confort_été,Isolation_toiture_(0/1),Protection_solaire_exterieure_(0/1),Inertie_lourde_(0/1),Deperditions_baies_vitrées,Deperditions_enveloppe,Déperditions_murs,Deperditions_planchers_bas,Deperditions_planchers_hauts,Déperditions_ponts_thermiques,Déperditions_portes,Déperditions_renouvellement_air,Qualité_isolation_enveloppe,Qualité_isolation_menuiseries,Qualité_isolation_murs,Qualité_isolation_plancher_bas,Ubat_W/m²_K,Qualité_isolation_plancher_haut_toit_terrase,Qualité_isolation_plancher_haut_comble_aménagé,Qualité_isolation_plancher_haut_comble_perdu,Apports_internes_saison_chauffe_,Apports_internes_saison_froid,Apports_solaires_saison_chauffe,Apports_solaires_saison_froid,Besoin_chauffage,Besoin_ECS,Besoin_refroidissement,Besoin_refroidissement_dépensier,Type_énergie_principale_chauffage,Conso_chauffage_installation_chauffage_n°1,Conso_chauffage_dépensier_installatio

In [48]:
print(df2.columns.tolist())

['Unnamed: 0', 'N°DPE', 'Date_réception_DPE', 'Date_établissement_DPE', 'Date_visite_diagnostiqueur', 'Modèle_DPE', 'N°_DPE_remplacé', 'Date_fin_validité_DPE', 'Version_DPE', 'N°_DPE_immeuble_associé', 'Appartement_non_visité_(0/1)', 'Méthode_application_DPE', 'N°_immatriculation_copropriété', 'Invariant_fiscal_logement', 'N°_RPLS_logement', 'Etiquette_GES', 'Etiquette_DPE', 'Année_construction', 'Type_bâtiment', 'Type_installation_chauffage', 'Type_installation_ECS_(général)', 'Période_construction', 'Hauteur_sous-plafond', 'Nombre_appartement', 'Nombre_niveau_immeuble', 'Nombre_niveau_logement', 'Surface_habitable_immeuble', 'Surface_habitable_logement', 'Surface_tertiaire_immeuble', 'Classe_inertie_bâtiment', 'Typologie_logement', 'Position_logement_dans_immeuble', 'Classe_altitude', 'Zone_climatique_', 'Adresse_brute', 'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)', 'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN', 'Nom__rue_(BAN)', 'Coordonnée_carto

In [49]:
# checking that there are everything in Lyon city
print(df2['Nom__commune_(BAN)'].unique().tolist())

['LYON', 'Lyon', 'Lyon 1er Arrondissement', 'Lyon 2e Arrondissement', 'Lyon 3e Arrondissement', 'Lyon 4e Arrondissement', 'Lyon 5e Arrondissement', 'Lyon 6e Arrondissement', 'Lyon 7e Arrondissement', 'Lyon 8e Arrondissement', 'LYON 9E ARRONDISSEMENT', 'Lyon 9e Arrondissement']


In [50]:
print(df2['Code_INSEE_(BAN)'].unique().tolist())

[69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388, 69389]


In [51]:
df2.sample(10)

,Unnamed: 0,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),Méthode_application_DPE,N°_immatriculation_copropriété,Invariant_fiscal_logement,N°_RPLS_logement,Etiquette_GES,Etiquette_DPE,Année_construction,Type_bâtiment,Type_installation_chauffage,Type_installation_ECS_(général),Période_construction,Hauteur_sous-plafond,Nombre_appartement,Nombre_niveau_immeuble,Nombre_niveau_logement,Surface_habitable_immeuble,Surface_habitable_logement,Surface_tertiaire_immeuble,Classe_inertie_bâtiment,Typologie_logement,Position_logement_dans_immeuble,Classe_altitude,Zone_climatique_,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Complément_d'adresse_bâtiment,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Conso_5_usages_é_finale,Conso_5_usages/m²_é_finale,Conso_chauffage_é_finale,Conso_chauffage_dépensier_é_finale,Conso_éclairage_é_finale,Conso_ECS_é_finale,Conso_ECS_dépensier_é_finale,Conso_refroidissement_é_finale,Conso_refroidissement_dépensier_é_finale,Conso_auxiliaires_é_finale,Conso_5_usages_é_primaire,Conso_5_usages_par_m²_é_primaire,Conso_chauffage_é_primaire,Conso_chauffage_dépensier_é_primaire,Conso_éclairage_é_primaire,Conso_ECS_é_primaire,Conso_ECS_dépensier_é_primaire,Conso_refroidissement_é_primaire,Conso_refroidissement_dépensier_é_primaire,Conso_auxiliaires_é_primaire,Emission_GES_5_usages,Emission_GES_5_usages_par_m²,Emission_GES_chauffage,Emission_GES_chauffage_dépensier,Emission_GES_éclairage,Emission_GES_ECS,Emission_GES_ECS_dépensier,Emission_GES_refroidissement,Emission_GES_refroidissement_dépensier,Emission_GES_auxiliaires,Conso_5_usages_é_finale_énergie_n°1,Conso_chauffage_é_finale_énergie_n°1,Conso_ECS_é_finale_énergie_n°1,Coût_total_5_usages_énergie_n°1,Coût_chauffage_énergie_n°1,Coût_ECS_énergie_n°1,Emission_GES_5_usages_énergie_n°1,Emission_GES_chauffage_énergie_n°1,Emission_GES_ECS_énergie_n°1,Type_énergie_n°1,Conso_5_usages_é_finale_énergie_n°2,Conso_chauffage_é_finale_énergie_n°2,Conso_ECS_é_finale_énergie_n°2,Coût_total_5_usages_énergie_n°2,Coût_chauffage_énergie_n°2,Coût_ECS_énergie_n°2,Emission_GES_5_usages_énergie_n°2,Emission_GES_chauffage_énergie_n°2,Emission_GES_ECS_énergie_n°2,Type_énergie_n°2,Conso_5_usages_é_finale_énergie_n°3,Conso_chauffage_é_finale_énergie_n°3,Conso_ECS_é_finale_énergie_n°3,Coût_total_5_usages_énergie_n°3,Coût_chauffage_énergie_n°3,Coût_ECS_énergie_n°3,Emission_GES_5_usages_énergie_n°3,Emission_GES_chauffage_énergie_n°3,Emission_GES_ECS_énergie_n°3,Type_énergie_n°3,Coût_total_5_usages,Coût_chauffage,Coût_chauffage_dépensier,Coût_éclairage,Coût_ECS,Coût_ECS_dépensier,Coût_refroidissement,Coût_refroidissement_dépensier,Coût_auxiliaires,Logement_traversant_(0/1),Présence_brasseur_air_(0/1),Indicateur_confort_été,Isolation_toiture_(0/1),Protection_solaire_exterieure_(0/1),Inertie_lourde_(0/1),Deperditions_baies_vitrées,Deperditions_enveloppe,Déperditions_murs,Deperditions_planchers_bas,Deperditions_planchers_hauts,Déperditions_ponts_thermiques,Déperditions_portes,Déperditions_renouvellement_air,Qualité_isolation_enveloppe,Qualité_isolation_menuiseries,Qualité_isolation_murs,Qualité_isolation_plancher_bas,Ubat_W/m²_K,Qualité_isolation_plancher_haut_toit_terrase,Qualité_isolation_plancher_haut_comble_aménagé,Qualité_isolation_plancher_haut_comble_perdu,Apports_internes_saison_chauffe_,Apports_internes_saison_froid,Apports_solaires_saison_chauffe,Apports_solaires_saison_froid,Besoin_chauffage,Besoin_ECS,Besoin_refroidissement,Besoin_refroidissement_dépensier,Type_énergie_principale_chauffage,Conso_chauffage_installation_chauffage_n°1,Conso_chauffage_dépensier_installatio

In [52]:
# keeping only useful columns. Since the methodology have changed, we loose the orientation feature
col_to_keep = ['Date_établissement_DPE', 'Année_construction','Etiquette_DPE', 'Type_bâtiment', 'Période_construction', 'Surface_habitable_logement', 'Code_postal_(BAN)', 'Nom__rue_(BAN)', 'N°_voie_(BAN)', 'Nom_résidence', 'Cage_d\'escalier', 'Complément_d\'adresse_logement', 'N°_étage_appartement', 'Nombre_niveau_logement']

In [53]:
dfi2 = df2[col_to_keep]

In [55]:
dfi2['Etiquette_DPE'].unique() # all the values are normal

array(['E', 'D', 'C', 'B', 'G', 'F', 'A'], dtype=object)

In [57]:
dfi2['Type_bâtiment'].unique() # only flats and houses

array(['appartement', 'immeuble', 'maison'], dtype=object)

In [58]:
dfi2['Période_construction'].unique()

array(['1948-1974', 'avant 1948', '1989-2000', '2006-2012', '1975-1977',
       '1983-1988', '2001-2005', '1978-1982', '2013-2021', 'après 2021'],
      dtype=object)

In [59]:
# keeping only the year of the diagnostics
dfi2['Annee'] = dfi2['Date_établissement_DPE'].apply(lambda x: x.split('-')[0])

In [60]:
dfi2['Code_postal_(BAN)'] = dfi2['Code_postal_(BAN)'].astype(int)

In [61]:
#treating the building periods
def annee(ancons, percons) :
    if math.isnan(ancons) == True :
        if percons == 'avant 1948' :
            return 1947
        if percons == 'après 2021' :
            return 2022
        else :
            return (int(percons.split('-')[0])+int(percons.split('-')[1]))/2 # we take the mean of the period if the year of the building is not precisely mentioned
    else :
        return ancons


In [62]:
dfi2['Annee_construction'] = list(map(annee, dfi2['Année_construction'], dfi2['Période_construction']))

In [63]:
#get the badly written floor number in the file. Check if the word 'étage' is written in the column Complément_d'adresse_logement
dfi2['Etage_num'] = dfi2['Complément_d\'adresse_logement'].apply(lambda x: list(map(int, re.findall(r'\d+', str(x)))))
dfi2['Etage_wa'] = dfi2['Complément_d\'adresse_logement'].apply(lambda x: list(map(str, re.findall('étage', str(x).lower()))))
dfi2['Etage_na'] = dfi2['Complément_d\'adresse_logement'].apply(lambda x: list(map(str, re.findall('etage', str(x).lower()))))
dfi2.sample(10, random_state = 42)

,Date_établissement_DPE,Année_construction,Etiquette_DPE,Type_bâtiment,Période_construction,Surface_habitable_logement,Code_postal_(BAN),Nom__rue_(BAN),N°_voie_(BAN),Nom_résidence,Cage_d'escalier,Complément_d'adresse_logement,N°_étage_appartement,Nombre_niveau_logement,Annee,Annee_construction,Etage_num,Etage_wa,Etage_na
33676,2021-11-15,2009.0,D,appartement,2006-2012,17.9,69009,Rue Sergent Michel Berthet,39bis,NaN,NaN,Etage RDC; Porte 12,0.0,1.0,2021,2009.0,[12],[],[etage]
30264,2022-10-06,1998.0,C,appartement,1989-2000,77.3,69008,Avenue des Frères Lumière,146,NaN,NaN,NaN,3.0,1.0,2022,1998.0,[],[],[]
17090,2022-09-27,NaN,G,appartement,avant 1948,41.8,69005,Rue du Viel Renversé,6,NaN,NaN,"1er ETAGE, porte gauche sur coursive",0.0,1.0,2022,1947.0,[1],[],[etage]
31681,2022-10-22,NaN,E,appartement,1948-1974,80.2,69008,Avenue Paul Santy,47,NaN,NaN,1er étage,0.0,1.0,2022,1961.0,[1],[étage],[]
27517,2022-02-08,1947.0,E,appartement,avant 1948,69.0,69007,Rue de la Thibaudière,49,NaN,NaN,NaN,0.0,1.0,2022,1947.0,[],[],[]
1162,2022-04-11,1947.0,D,appartement,avant 1948,67.6,69001,Rue Pouteau,8,NaN,NaN,Etage 1,0.0,2.0,2022,1947.0,[1],[],[etage]
7403,2022-07-28,1975.0,C,appartement,1975-1977,100.2,69003,Rue Garibaldi,290,NaN,NaN,NaN,0.0,1.0,2022,1975.0,[],[],[]
4819,2022-07-07,NaN,D,appartement,avant 1948,22.4,69002,Rue Mercière,7,NaN,NaN,NaN,0.0,1.0,2022,1947.0,[],[],[]
24606,2022-08-18,NaN,C,appartement,2001-2005,82.9,69007,Rue Marie-Madeleine Fourcade,19,NaN,NaN,Appartement au 1er étage,0.0,1.0,2022,2003.0,[1],[étage],[]
31758,2022-07-10,NaN,F,appartement,1948-1974,57.0,69008,Impasse Ferret,3,NaN,NaN,NaN,0.0,2.0,2022,1961.0,[],[],[]


In [64]:
#if we find RDC = rez de chaussee
dfi2['Etage_RDC'] = dfi2['Complément_d\'adresse_logement'].apply(lambda x: re.findall('rdc', str(x).lower()))

In [65]:
# keeping the first detected number
dfi2['Etage_num'] = dfi2['Etage_num'].apply(lambda x: min(x) if len(x)>0 else None)
dfi2['Etage_num'] = pd.to_numeric(dfi2['Etage_num'])

In [66]:
def etage(netage, num, RDC, wa, na) :
    if netage > 0 :             # if the value in the column 'N°_étage_appartement' is already here, we already have the floor number
        return netage
    else :
        if math.isnan(num) == True and len(RDC)==0 : # if either the value in the column 'N°_étage_appartement' is not here nor we detected RDC, we have no floor number
            return float('NaN')
        if len(RDC) >= 1 :                      # if we detected RDC, then the floor is rez-de-chaussée
            return 0
        else :
            if num > 11 :                   # if we get a number > 11, then we assume that it is not a floor number and we return NaN
                return float('NaN')
            else :                          # if we get a number <= 11, then we assume that it is the floor number provided that the word 'étage' or 'etage' was detected
                if len(wa) >= 1 or len(na) >=1 :
                    return num
                else :
                    return float('NaN')

In [67]:
dfi2['Etage'] = list(map(etage, dfi2['N°_étage_appartement'], dfi2['Etage_num'], dfi2['Etage_RDC'], dfi2['Etage_wa'], dfi2['Etage_na']))
dfi2.sample(10, random_state = 42)

,Date_établissement_DPE,Année_construction,Etiquette_DPE,Type_bâtiment,Période_construction,Surface_habitable_logement,Code_postal_(BAN),Nom__rue_(BAN),N°_voie_(BAN),Nom_résidence,Cage_d'escalier,Complément_d'adresse_logement,N°_étage_appartement,Nombre_niveau_logement,Annee,Annee_construction,Etage_num,Etage_wa,Etage_na,Etage_RDC,Etage
33676,2021-11-15,2009.0,D,appartement,2006-2012,17.9,69009,Rue Sergent Michel Berthet,39bis,NaN,NaN,Etage RDC; Porte 12,0.0,1.0,2021,2009.0,12.0,[],[etage],[rdc],0.0
30264,2022-10-06,1998.0,C,appartement,1989-2000,77.3,69008,Avenue des Frères Lumière,146,NaN,NaN,NaN,3.0,1.0,2022,1998.0,NaN,[],[],[],3.0
17090,2022-09-27,NaN,G,appartement,avant 1948,41.8,69005,Rue du Viel Renversé,6,NaN,NaN,"1er ETAGE, porte gauche sur coursive",0.0,1.0,2022,1947.0,1.0,[],[etage],[],1.0
31681,2022-10-22,NaN,E,appartement,1948-1974,80.2,69008,Avenue Paul Santy,47,NaN,NaN,1er étage,0.0,1.0,2022,1961.0,1.0,[étage],[],[],1.0
27517,2022-02-08,1947.0,E,appartement,avant 1948,69.0,69007,Rue de la Thibaudière,49,NaN,NaN,NaN,0.0,1.0,2022,1947.0,NaN,[],[],[],NaN
1162,2022-04-11,1947.0,D,appartement,avant 1948,67.6,69001,Rue Pouteau,8,NaN,NaN,Etage 1,0.0,2.0,2022,1947.0,1.0,[],[etage],[],1.0
7403,2022-07-28,1975.0,C,appartement,1975-1977,100.2,69003,Rue Garibaldi,290,NaN,NaN,NaN,0.0,1.0,2022,1975.0,NaN,[],[],[],NaN
4819,2022-07-07,NaN,D,appartement,avant 1948,22.4,69002,Rue Mercière,7,NaN,NaN,NaN,0.0,1.0,2022,1947.0,NaN,[],[],[],NaN
24606,2022-08-18,NaN,C,appartement,2001-2005,82.9,69007,Rue Marie-Madeleine Fourcade,19,NaN,NaN,Appartement au 1er étage,0.0,1.0,2022,2003.0,1.0,[étage],[],[],1.0
31758,2022-07-10,NaN,F,appartement,1948-1974,57.0,69008,Impasse Ferret,3,NaN,NaN,NaN,0.0,2.0,2022,1961.0,NaN,[],[],[],NaN


In [68]:
# reducing the dataset with only the useful columns
col_to_keep = ['Annee', 'Etiquette_DPE', 'Type_bâtiment', 'Annee_construction', 'Surface_habitable_logement', 'N°_voie_(BAN)', 'Nom__rue_(BAN)', 'Code_postal_(BAN)', 'Etage', 'Nombre_niveau_logement']


In [69]:
dfi2 = dfi2[col_to_keep]
dfi2['orientation'] = np.nan # impossible to find orientation with this new dataset because the methodology has changed

In [71]:
dfi2.groupby(['Annee']).size() # number of diagnostics are normal compared to the previous years

Annee
2021     8829
2022    27917
dtype: int64

In [72]:
# giving the same column names as the first dataset
dfi2.columns = list_columns_dpe

In [73]:
dfi2.sample(10)

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
22866,2021,C,appartement,1989.0,88.0,21,Rue Père Chevrier,69007,6.0,1.0,NaN
18874,2021,C,appartement,1999.0,99.2,96,Rue Boileau,69006,3.0,1.0,NaN
17667,2022,C,appartement,1980.0,87.5,18,Rue Soeur Janin,69005,NaN,13.0,NaN
4057,2022,D,appartement,1948.0,239.5,12,Quai Tilsitt,69002,NaN,1.0,NaN
20879,2022,F,appartement,1961.0,104.0,9,Rue Garibaldi,69006,4.0,1.0,NaN
2829,2021,C,appartement,1989.0,120.8,9,Rue dâ€™Enghien,69002,5.0,1.0,NaN
17223,2022,D,appartement,1985.0,82.7,21ter,Avenue du Point du Jour,69005,0.0,1.0,NaN
36232,2022,C,appartement,2009.0,52.4,37,Rue Louis Juttet,69009,2.0,1.0,NaN
29099,2022,D,appartement,1974.0,80.0,106,Avenue Paul Santy,69008,NaN,1.0,NaN
34142,2022,D,appartement,1994.5,24.0,16,Rue Chinard,69009,NaN,1.0,NaN


# Dataset after july 2021, only new accomodation

In [75]:
# we do exactly the same preprocessings as for the second dataset
df3 = pd.read_csv('../src/dpe-lyon-neuf-21.csv')

In [76]:
df3.shape

(3354, 220)

In [77]:
df3.head()

,Unnamed: 0,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),Méthode_application_DPE,N°_immatriculation_copropriété,N°_RPLS_logement,Invariant_fiscal_logement,Etiquette_DPE,Etiquette_GES,Type_bâtiment,Type_installation_chauffage,Type_installation_ECS_(général),Hauteur_sous-plafond,Nombre_appartement,Nombre_niveau_immeuble,Nombre_niveau_logement,Surface_habitable_immeuble,Surface_tertiaire_immeuble,Surface_habitable_logement,Classe_inertie_bâtiment,Typologie_logement,Position_logement_dans_immeuble,Classe_altitude,Zone_climatique_,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Complément_d'adresse_bâtiment,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Conso_5_usages_é_finale,Conso_5_usages/m²_é_finale,Conso_chauffage_é_finale,Conso_chauffage_dépensier_é_finale,Conso_éclairage_é_finale,Conso_ECS_é_finale,Conso_ECS_dépensier_é_finale,Conso_refroidissement_é_finale,Conso_refroidissement_dépensier_é_finale,Conso_auxiliaires_é_finale,Conso_5_usages_é_primaire,Conso_5_usages_par_m²_é_primaire,Conso_chauffage_é_primaire,Conso_chauffage_dépensier_é_primaire,Conso_éclairage_é_primaire,Conso_ECS_é_primaire,Conso_ECS_dépensier_é_primaire,Conso_refroidissement_é_primaire,Conso_refroidissement_dépensier_é_primaire,Conso_auxiliaires_é_primaire,Emission_GES_5_usages,Emission_GES_5_usages_par_m²,Emission_GES_chauffage,Emission_GES_chauffage_dépensier,Emission_GES_éclairage,Emission_GES_ECS,Emission_GES_ECS_dépensier,Emission_GES_refroidissement,Emission_GES_refroidissement_dépensier,Emission_GES_auxiliaires,Conso_5_usages_énergie_n°1,Conso_chauffage_énergie_n°1,Conso_ECS_énergie_n°1,Coût_5_usages_énergie_n°1,Coût_chauffage_énergie_n°1,Coût_ECS_énergie_n°1,Emission_GES_5_usages_énergie_n°1,Emission_GES_chauffage_par_énergie_n°1,Emission_GES_ECS_par_énergie_n°1,Type_énergie_n°1,Conso_5_usages_énergie_n°2,Conso_chauffage_énergie_n°2,Conso_ECS_énergie_n°2,Coût_5_usages_énergie_n°2,Coût_chauffage_énergie_n°2,Coût_ECS_énergie_n°2,Emission_GES_5_usages_énergie_n°2,Emission_GES_chauffage_par_énergie_n°2,Emission_GES_ECS_par_énergie_n°2,Type_énergie_n°2,Conso_5_usages_énergie_n°3,Conso_chauffage_énergie_n°3,Conso_ECS_énergie_n°3,Coût_5_usages_énergie_n°3,Coût_chauffage_énergie_n°3,Coût_ECS_énergie_n°3,Emission_GES_5_usages_énergie_n°3,Emission_GES_chauffage_par_énergie_n°3,Emission_GES_ECS_par_énergie_n°3,Type_énergie_n°3,Coût_total_5_usages,Coût_chauffage,Coût_chauffage_dépensier,Coût_éclairage,Coût_ECS_dépensier,Coût_ECS,Coût_refroidissement,Coût_refroidissement_dépensier,Coût_auxiliaires,Logement_traversant_(0/1),Présence_brasseur_air_(0/1),Indicateur_confort_été,Isolation_toiture_(0/1),Protection_solaire_exterieure_(0/1),Inertie_lourde_(0/1),Deperditions_baies_vitrées,Deperditions_enveloppe,Déperditions_murs,Deperditions_planchers_bas,Deperditions_planchers_hauts,Déperditions_ponts_thermiques,Déperditions_portes,Déperditions_renouvellement_air,Qualité_isolation_enveloppe,Qualité_isolation_menuiseries,Qualité_isolation_murs,Qualité_isolation_plancher_bas,Ubat_W/m²_K,Qualité_isolation_plancher_haut_comble_aménagé,Besoin_chauffage,Besoin_ECS,Besoin_refroidissement,Conso_chauffage_installation_chauffage_n°1,Description_installation_chauffage_n°1,Configuration_installation_chauffage_n°1,Type_installation_chauffage_n°1,Surface_chauffée_installation_chauffage_n°1,Type_émetteur_installation_chauffage_n°1,Conso_chauffage_générateur_n°1_installation_n°1,Type_générateur_n°2_installation_n°1,Conso_chauffage_dépensier_générateur_n°1_installation_n°1,Type_énergie_générateur_n°1_installation_n°1,Type_générateur_n°1_installation_n°1,Usage_généra

In [78]:
print(df3.columns.tolist())

['Unnamed: 0', 'N°DPE', 'Date_réception_DPE', 'Date_établissement_DPE', 'Date_visite_diagnostiqueur', 'Modèle_DPE', 'N°_DPE_remplacé', 'Date_fin_validité_DPE', 'Version_DPE', 'N°_DPE_immeuble_associé', 'Appartement_non_visité_(0/1)', 'Méthode_application_DPE', 'N°_immatriculation_copropriété', 'N°_RPLS_logement', 'Invariant_fiscal_logement', 'Etiquette_DPE', 'Etiquette_GES', 'Type_bâtiment', 'Type_installation_chauffage', 'Type_installation_ECS_(général)', 'Hauteur_sous-plafond', 'Nombre_appartement', 'Nombre_niveau_immeuble', 'Nombre_niveau_logement', 'Surface_habitable_immeuble', 'Surface_tertiaire_immeuble', 'Surface_habitable_logement', 'Classe_inertie_bâtiment', 'Typologie_logement', 'Position_logement_dans_immeuble', 'Classe_altitude', 'Zone_climatique_', 'Adresse_brute', 'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)', 'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN', 'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)', 'Coordonnée_cartographique

In [79]:
print(df3['Nom__commune_(BAN)'].unique().tolist())

['Lyon']


In [80]:
print(df3['Code_INSEE_(BAN)'].unique().tolist())

[69123, 69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388, 69389]


In [81]:
df3.sample(10)

,Unnamed: 0,N°DPE,Date_réception_DPE,Date_établissement_DPE,Date_visite_diagnostiqueur,Modèle_DPE,N°_DPE_remplacé,Date_fin_validité_DPE,Version_DPE,N°_DPE_immeuble_associé,Appartement_non_visité_(0/1),Méthode_application_DPE,N°_immatriculation_copropriété,N°_RPLS_logement,Invariant_fiscal_logement,Etiquette_DPE,Etiquette_GES,Type_bâtiment,Type_installation_chauffage,Type_installation_ECS_(général),Hauteur_sous-plafond,Nombre_appartement,Nombre_niveau_immeuble,Nombre_niveau_logement,Surface_habitable_immeuble,Surface_tertiaire_immeuble,Surface_habitable_logement,Classe_inertie_bâtiment,Typologie_logement,Position_logement_dans_immeuble,Classe_altitude,Zone_climatique_,Adresse_brute,Nom__commune_(BAN),Code_INSEE_(BAN),N°_voie_(BAN),Identifiant__BAN,Adresse_(BAN),Code_postal_(BAN),Score_BAN,Nom__rue_(BAN),Coordonnée_cartographique_X_(BAN),Coordonnée_cartographique_Y_(BAN),Code_postal_(brut),N°_étage_appartement,Nom_résidence,Complément_d'adresse_bâtiment,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Conso_5_usages_é_finale,Conso_5_usages/m²_é_finale,Conso_chauffage_é_finale,Conso_chauffage_dépensier_é_finale,Conso_éclairage_é_finale,Conso_ECS_é_finale,Conso_ECS_dépensier_é_finale,Conso_refroidissement_é_finale,Conso_refroidissement_dépensier_é_finale,Conso_auxiliaires_é_finale,Conso_5_usages_é_primaire,Conso_5_usages_par_m²_é_primaire,Conso_chauffage_é_primaire,Conso_chauffage_dépensier_é_primaire,Conso_éclairage_é_primaire,Conso_ECS_é_primaire,Conso_ECS_dépensier_é_primaire,Conso_refroidissement_é_primaire,Conso_refroidissement_dépensier_é_primaire,Conso_auxiliaires_é_primaire,Emission_GES_5_usages,Emission_GES_5_usages_par_m²,Emission_GES_chauffage,Emission_GES_chauffage_dépensier,Emission_GES_éclairage,Emission_GES_ECS,Emission_GES_ECS_dépensier,Emission_GES_refroidissement,Emission_GES_refroidissement_dépensier,Emission_GES_auxiliaires,Conso_5_usages_énergie_n°1,Conso_chauffage_énergie_n°1,Conso_ECS_énergie_n°1,Coût_5_usages_énergie_n°1,Coût_chauffage_énergie_n°1,Coût_ECS_énergie_n°1,Emission_GES_5_usages_énergie_n°1,Emission_GES_chauffage_par_énergie_n°1,Emission_GES_ECS_par_énergie_n°1,Type_énergie_n°1,Conso_5_usages_énergie_n°2,Conso_chauffage_énergie_n°2,Conso_ECS_énergie_n°2,Coût_5_usages_énergie_n°2,Coût_chauffage_énergie_n°2,Coût_ECS_énergie_n°2,Emission_GES_5_usages_énergie_n°2,Emission_GES_chauffage_par_énergie_n°2,Emission_GES_ECS_par_énergie_n°2,Type_énergie_n°2,Conso_5_usages_énergie_n°3,Conso_chauffage_énergie_n°3,Conso_ECS_énergie_n°3,Coût_5_usages_énergie_n°3,Coût_chauffage_énergie_n°3,Coût_ECS_énergie_n°3,Emission_GES_5_usages_énergie_n°3,Emission_GES_chauffage_par_énergie_n°3,Emission_GES_ECS_par_énergie_n°3,Type_énergie_n°3,Coût_total_5_usages,Coût_chauffage,Coût_chauffage_dépensier,Coût_éclairage,Coût_ECS_dépensier,Coût_ECS,Coût_refroidissement,Coût_refroidissement_dépensier,Coût_auxiliaires,Logement_traversant_(0/1),Présence_brasseur_air_(0/1),Indicateur_confort_été,Isolation_toiture_(0/1),Protection_solaire_exterieure_(0/1),Inertie_lourde_(0/1),Deperditions_baies_vitrées,Deperditions_enveloppe,Déperditions_murs,Deperditions_planchers_bas,Deperditions_planchers_hauts,Déperditions_ponts_thermiques,Déperditions_portes,Déperditions_renouvellement_air,Qualité_isolation_enveloppe,Qualité_isolation_menuiseries,Qualité_isolation_murs,Qualité_isolation_plancher_bas,Ubat_W/m²_K,Qualité_isolation_plancher_haut_comble_aménagé,Besoin_chauffage,Besoin_ECS,Besoin_refroidissement,Conso_chauffage_installation_chauffage_n°1,Description_installation_chauffage_n°1,Configuration_installation_chauffage_n°1,Type_installation_chauffage_n°1,Surface_chauffée_installation_chauffage_n°1,Type_émetteur_installation_chauffage_n°1,Conso_chauffage_générateur_n°1_installation_n°1,Type_générateur_n°2_installation_n°1,Conso_chauffage_dépensier_générateur_n°1_installation_n°1,Type_énergie_générateur_n°1_installation_n°1,Type_générateur_n°1_installation_n°1,Usage_généra

In [82]:
col_to_keep = ['Date_établissement_DPE','Etiquette_DPE', 'Type_bâtiment', 'Surface_habitable_logement', 'N°_voie_(BAN)', 'Nom__rue_(BAN)', 'Code_postal_(BAN)', 'Complément_d\'adresse_logement', 'N°_étage_appartement', 'Nombre_niveau_logement']

In [83]:
dfi3 = df3[col_to_keep]

In [84]:
dfi3['Etiquette_DPE'].unique()

array(['C', 'B', 'A', 'D', 'E', 'G', 'F'], dtype=object)

In [85]:
dfi3['Type_bâtiment'].unique()

array(['appartement', 'maison', 'immeuble'], dtype=object)

In [86]:
dfi3['Annee'] = dfi3['Date_établissement_DPE'].apply(lambda x: x.split('-')[0])

In [87]:
dfi3['Code_postal_(BAN)'] = dfi3['Code_postal_(BAN)'].astype(int)

In [88]:
#get the badly written floor number in the file. Check if the word 'étage' is written in the column Complément_d'adresse_logement
dfi3['Etage_num'] = dfi3['Complément_d\'adresse_logement'].apply(lambda x: list(map(int, re.findall(r'\d+', str(x)))))
dfi3['Etage_wa'] = dfi3['Complément_d\'adresse_logement'].apply(lambda x: list(map(str, re.findall('étage', str(x).lower()))))
dfi3['Etage_na'] = dfi3['Complément_d\'adresse_logement'].apply(lambda x: list(map(str, re.findall('etage', str(x).lower()))))
dfi3.sample(10, random_state = 42)

,Date_établissement_DPE,Etiquette_DPE,Type_bâtiment,Surface_habitable_logement,N°_voie_(BAN),Nom__rue_(BAN),Code_postal_(BAN),Complément_d'adresse_logement,N°_étage_appartement,Nombre_niveau_logement,Annee,Etage_num,Etage_wa,Etage_na
414,2022-09-27,C,appartement,114.3,11,Rue Bonnand,69003,Apt 7,0.0,NaN,2022,[7],[],[]
314,2022-06-15,B,appartement,63.1,24,Avenue Lacassagne,69003,NaN,0.0,NaN,2022,[],[],[]
2381,2022-05-12,B,appartement,19.9,17,Rue Miriam Makeba,69008,NaN,NaN,1.0,2022,[],[],[]
3236,2022-09-22,C,appartement,20.3,4,Passage du Coteau,69009,Bat. N/A C607,0.0,NaN,2022,[607],[],[]
2548,2022-02-15,B,appartement,39.4,6,Rue Colonel Arnaud Beltrame,69008,NaN,NaN,1.0,2022,[],[],[]
3304,2022-09-22,C,appartement,20.1,4,Passage du Coteau,69009,Bat. N/A C604,0.0,NaN,2022,[604],[],[]
2431,2022-05-12,C,appartement,19.8,17,Rue Miriam Makeba,69008,NaN,NaN,1.0,2022,[],[],[]
1344,2022-03-25,A,appartement,49.3,7,Place Vaclav Havel,69007,NaN,NaN,1.0,2022,[],[],[]
3150,2022-09-22,C,appartement,20.0,6,Passage du Coteau,69009,Bat. B B410,0.0,NaN,2022,[410],[],[]
727,2022-03-14,B,appartement,88.7,NaN,NaN,69006,NaN,0.0,NaN,2022,[],[],[]


In [89]:
#if RDC detected
dfi3['Etage_RDC'] = dfi3['Complément_d\'adresse_logement'].apply(lambda x: re.findall('rdc', str(x).lower()))

In [90]:
dfi3['Etage_num'] = dfi3['Etage_num'].apply(lambda x: min(x) if len(x)>0 else None)
dfi3['Etage_num'] = pd.to_numeric(dfi3['Etage_num'])

In [91]:
def etage(netage, num, RDC, wa, na) :
    if netage > 0 :             
        return netage
    else :
        if math.isnan(num) == True and len(RDC)==0 : 
            return float('NaN')
        if len(RDC) >= 1 :                     
            return 0
        else :
            if num > 11 :                   
                return float('NaN')
            else :                          
                if len(wa) >= 1 or len(na) >=1 :
                    return num
                else :
                    return float('NaN')

In [92]:
dfi3['Etage'] = list(map(etage, dfi3['N°_étage_appartement'], dfi3['Etage_num'], dfi3['Etage_RDC'], dfi3['Etage_wa'], dfi3['Etage_na']))
dfi3.sample(10, random_state = 42)

,Date_établissement_DPE,Etiquette_DPE,Type_bâtiment,Surface_habitable_logement,N°_voie_(BAN),Nom__rue_(BAN),Code_postal_(BAN),Complément_d'adresse_logement,N°_étage_appartement,Nombre_niveau_logement,Annee,Etage_num,Etage_wa,Etage_na,Etage_RDC,Etage
414,2022-09-27,C,appartement,114.3,11,Rue Bonnand,69003,Apt 7,0.0,NaN,2022,7.0,[],[],[],NaN
314,2022-06-15,B,appartement,63.1,24,Avenue Lacassagne,69003,NaN,0.0,NaN,2022,NaN,[],[],[],NaN
2381,2022-05-12,B,appartement,19.9,17,Rue Miriam Makeba,69008,NaN,NaN,1.0,2022,NaN,[],[],[],NaN
3236,2022-09-22,C,appartement,20.3,4,Passage du Coteau,69009,Bat. N/A C607,0.0,NaN,2022,607.0,[],[],[],NaN
2548,2022-02-15,B,appartement,39.4,6,Rue Colonel Arnaud Beltrame,69008,NaN,NaN,1.0,2022,NaN,[],[],[],NaN
3304,2022-09-22,C,appartement,20.1,4,Passage du Coteau,69009,Bat. N/A C604,0.0,NaN,2022,604.0,[],[],[],NaN
2431,2022-05-12,C,appartement,19.8,17,Rue Miriam Makeba,69008,NaN,NaN,1.0,2022,NaN,[],[],[],NaN
1344,2022-03-25,A,appartement,49.3,7,Place Vaclav Havel,69007,NaN,NaN,1.0,2022,NaN,[],[],[],NaN
3150,2022-09-22,C,appartement,20.0,6,Passage du Coteau,69009,Bat. B B410,0.0,NaN,2022,410.0,[],[],[],NaN
727,2022-03-14,B,appartement,88.7,NaN,NaN,69006,NaN,0.0,NaN,2022,NaN,[],[],[],NaN


In [93]:
dfi3['Annee_construction'] = dfi3['Annee'] # hypothesis : year of building missing but choice only between 2021 and 2022 => we assume that the year of building = year of diagnostic

In [94]:
col_to_keep = ['Annee', 'Etiquette_DPE', 'Type_bâtiment', 'Annee_construction', 'Surface_habitable_logement', 'N°_voie_(BAN)', 'Nom__rue_(BAN)', 'Code_postal_(BAN)', 'Etage', 'Nombre_niveau_logement']

In [95]:
dfi3 = dfi3[col_to_keep]
dfi3['orientation'] = np.nan # impossible to find orientation with this new dataset

In [96]:
dfi3.groupby(['Annee']).size()

Annee
2021    1024
2022    2330
dtype: int64

In [97]:
dfi3.columns = list_columns_dpe

In [98]:
dfi3.sample(10)

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
1584,2022,B,appartement,2022,93.0,6,Rue Colonel Arnaud Beltrame,69008,NaN,1.0,NaN
1431,2021,B,appartement,2021,44.7,NaN,NaN,69007,NaN,1.0,NaN
423,2022,A,appartement,2022,28.2,114,Rue Antoine Charial,69003,NaN,1.0,NaN
2909,2022,C,appartement,2022,19.6,4,Passage du Coteau,69009,NaN,NaN,NaN
721,2022,C,appartement,2022,89.5,NaN,Rue Notre-Dame,69006,NaN,1.0,NaN
2923,2022,C,appartement,2022,23.1,15,Rue Transversale,69009,NaN,7.0,NaN
1680,2022,B,appartement,2022,60.9,6,Rue Colonel Arnaud Beltrame,69008,NaN,1.0,NaN
1230,2022,A,appartement,2022,153.8,7,Place Vaclav Havel,69007,NaN,1.0,NaN
1237,2021,C,appartement,2021,80.8,NaN,NaN,69007,NaN,1.0,NaN
2974,2022,C,appartement,2022,21.2,8,Passage du Coteau,69009,NaN,NaN,NaN


# Concatenate 3 datasets into 1 unique dataset

In [99]:
df_dpe = pd.DataFrame()
df_dpe = pd.concat([dfi1, dfi2, dfi3], ignore_index=True)

In [100]:
print(dfi1.shape[0]+dfi2.shape[0]+dfi3.shape[0] == df_dpe.shape[0])
print(df_dpe.shape)

True
(71268, 11)


In [101]:
df_dpe.sample(10, random_state=42)

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
28960,2015,C,NaN,2006,43.00,NaN,"nan Rdc - Lot 4, 7, rue de la Métallurgie",69007,NaN,0.0,None
16132,2019,C,NaN,1989,133.00,NaN,nan 23 CHEMIN DE MONTPELLAS,69000,NaN,1.0,EO
31844,2022,D,appartement,1949.0,71.80,2,Rue Rivet,69001,1.0,2.0,NaN
22794,2020,D,NaN,1996,69.34,NaN,nan 84 Rue du Pensionnat,69000,NaN,1.0,S
32505,2022,E,appartement,1947.0,78.90,28,Rue Lanterne,69001,NaN,1.0,NaN
48111,2022,D,appartement,1947.0,37.00,120,Rue Saint Georges,69005,1.0,1.0,NaN
17172,2019,E,NaN,2010,51.00,NaN,nan 16 rue des Tuileries,69000,NaN,4.0,EO
39327,2021,D,appartement,1997.0,29.50,3,Avenue Lacassagne,69003,5.0,1.0,NaN
70924,2022,C,appartement,2022,19.30,6,Passage du Coteau,69009,NaN,NaN,NaN
36941,2021,D,appartement,1973.0,86.70,67,Rue Baraban,69003,8.0,1.0,NaN


In [102]:
df_dpe.describe(include='all')

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe
count,71268,71268,43142,71268.0,70928.000000,44076,69872,71268.000000,23285.000000,69226.000000,14745
unique,13,7,15,170.0,NaN,974,22256,NaN,NaN,NaN,3
top,2022,D,appartement,1947.0,NaN,4,Avenue Barthélémy Buyer,NaN,NaN,NaN,EO
freq,30247,25005,39142,22695.0,NaN,1590,694,NaN,NaN,NaN,7500
mean,NaN,NaN,NaN,NaN,83.802234,NaN,NaN,69003.524148,2.809835,1.396498,NaN
std,NaN,NaN,NaN,NaN,321.932374,NaN,NaN,3.259054,1.985992,2.229624,NaN
min,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,69000.000000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,40.800000,NaN,NaN,69000.000000,1.000000,1.000000,NaN
50%,NaN,NaN,NaN,NaN,59.800000,NaN,NaN,69003.000000,3.000000,1.000000,NaN
75%,NaN,NaN,NaN,NaN,79.090000,NaN,NaN,69007.000000,4.000000,1.000000,NaN


In [103]:
# building 'adresse' column to request the BAN API (French official API for normalizing and geocoding adresses)
def adresse(num, rue) :
    if pd.notnull(num) and pd.notnull(rue) :
        return num + ' ' + rue
    if pd.notnull(num)==False :
        return rue
    if pd.notnull(rue)==False :
        return num

In [104]:
df_dpe['num_rue_dpe'] = df_dpe['num_rue_dpe'].astype(str)
df_dpe['rue_dpe'] = df_dpe['rue_dpe'].astype(str)
df_dpe['adresse_dpe'] = list(map(adresse, df_dpe['num_rue_dpe'], df_dpe['rue_dpe'])) # concatenate and remove nan

In [105]:
df_dpe.head()

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe
0,2013,G,Habitation (Parties privatives d'immeuble col,1995,19.28,8,rue de l'Epargne,69000,1.0,0.0,None,8 rue de l'Epargne
1,2013,E,Habitation (parties privatives d immeuble col,1974,68.00,29,RUE RENE LEYNAUD,69000,3.0,1.0,None,29 RUE RENE LEYNAUD
2,2013,E,Habitation (parties privatives d immeuble col,2000,32.00,27,RUE DES GRANGES,69000,1.0,1.0,S,27 RUE DES GRANGES
3,2013,D,NaN,2000,38.23,NC,nan 60 Boulevard Pinel Bâtiment B,69000,NaN,1.0,S,NC nan 60 Boulevard Pinel Bâtiment B
4,2013,E,Habitation (parties privatives d immeuble col,1974,66.11,137,RUE de Montagny,69000,2.0,1.0,None,137 RUE de Montagny


In [106]:
# preparing threading to request API
cols = df_dpe.columns.to_list()
size = 6000         # my personal computer deals with 12 threads (71 268 rows / 12 = 6000)
list_threads = np.array_split(df_dpe, len(df_dpe) // size)
list_df_threads = [pd.DataFrame(list_threads[i], columns = cols) for i in range(len(list_threads))]

In [107]:
#get geokeys by requesting the API with threads of size 'size'
def get_geokeys(item) :
    item['geokey'] = np.nan
    for i in item.index.to_list() :
        param = {'q' : item.at[i,'adresse_dpe'], 'postcode' : item.at[i,'code_postal_dpe']}
        r = requests.get('https://api-adresse.data.gouv.fr/search/', params = param)
        result = json.loads(r.content)
        try :
            item.at[i,'geokey'] = result['features'][0]['properties']['id']
        except :
            item.at[i,'geokey'] = 0

In [108]:
with concurrent.futures.ThreadPoolExecutor() as executor :
    executor.map(get_geokeys, list_df_threads)

In [109]:
df_dpe_fi = pd.DataFrame()
for item in list_df_threads :
    df_dpe_fi = pd.concat([df_dpe_fi, item])

In [110]:
df_dpe_fi.sample(30, random_state=42)

,annee_dpe,diag_dpe,type_bat_dpe,annee_construction_dpe,surface_habitable_dpe,num_rue_dpe,rue_dpe,code_postal_dpe,etage_dpe,nbe_niveaux_dpe,orientation_dpe,adresse_dpe,geokey
28960,2015,C,NaN,2006,43.00,nan,"nan Rdc - Lot 4, 7, rue de la Métallurgie",69007,NaN,0.0,None,"nan nan Rdc - Lot 4, 7, rue de la Métallurgie",69387_4230_00007
16132,2019,C,NaN,1989,133.00,nan,nan 23 CHEMIN DE MONTPELLAS,69000,NaN,1.0,EO,nan nan 23 CHEMIN DE MONTPELLAS,0.0
31844,2022,D,appartement,1949.0,71.80,2,Rue Rivet,69001,1.0,2.0,NaN,2 Rue Rivet,69381_6050_00002
22794,2020,D,NaN,1996,69.34,nan,nan 84 Rue du Pensionnat,69000,NaN,1.0,S,nan nan 84 Rue du Pensionnat,0.0
32505,2022,E,appartement,1947.0,78.90,28,Rue Lanterne,69001,NaN,1.0,NaN,28 Rue Lanterne,69381_4140_00028
48111,2022,D,appartement,1947.0,37.00,120,Rue Saint Georges,69005,1.0,1.0,NaN,120 Rue Saint Georges,69385_6365_00120
17172,2019,E,NaN,2010,51.00,nan,nan 16 rue des Tuileries,69000,NaN,4.0,EO,nan nan 16 rue des Tuileries,0.0
39327,2021,D,appartement,1997.0,29.50,3,Avenue Lacassagne,69003,5.0,1.0,NaN,3 Avenue Lacassagne,69383_4080_00003
70924,2022,C,appartement,2022,19.30,6,Passage du Coteau,69009,NaN,NaN,NaN,6 Passage du Coteau,NaN
36941,2021,D,appartement,1973.0,86.70,67,Rue Baraban,69003,8.0,1.0,NaN,67 Rue Baraban,69383_0645_00067


In [107]:
df_dpe_fi.to_csv('df_dpe_geokey_221129.csv')